In [1]:
from diskcache_class import db
from lda_mallet import *
from pre_processing import *
from os import walk
from tqdm import tqdm

[nltk_data] Downloading package stopwords to /Users/das-
[nltk_data]     lab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/das-lab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/das-lab/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/das-lab/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [53]:
#we are going to run this on 4 different dataset 1. EDML 2. AP 3. Newsgroup 4.Covid-Tweet


############################## reading data for ED corpus
# datafolder = './data/ed_recovery_formatted/Excel'
# #datafolder = 'ed_recovery_topicmodel'
# df = pd.DataFrame(columns=['url','type','photo','date','tags','notes','text','photo_url','reblogged','blog_name'])

# for dirpath,dirnames,filenames in walk(datafolder):
#   for filename in filenames:
#     if filename.endswith('.xlsx'):
#       t = pd.read_excel(datafolder+'/'+filename,names=['url','type','photo','date','tags','notes','text','photo_url','reblogged'])
#       blog_name = t.iloc[0,0].split(':')[1]
#       t['blog_name'] = blog_name
#       df = df.append(t.iloc[3:,:],ignore_index=True)
#       print('blog:{0}   with posts:{1}    and reblogs:{2}  '.format(filename,len(t),len(t[t.reblogged=='yes'])))


# print('number of blogs: {0} - number of posts: {1}'.format(len(set(df.blog_name)),len(df)))
# print('out of {0} documents, {1} are reblogged.'.format(len(df),len(df[df.reblogged == 'yes'])))

# #finding reblogged texts
# texts = sorted(df.text) #sort them to keep smallest post (perhaps original one) at first
# re_texts = []

# while len(texts):
#   t = [texts.pop(0)]#pop first text and find it!
#   if t[0] == ' ' or len(t[0].split())<3: 
#     continue #almost nothing to look
#   i = 0
#   while i<len(texts):
#     if t[0] in texts[i]:
#       t.append(texts.pop(i))
#     else:
#       i += 1
#   if len(t) > 1:
#     re_texts.append(t)


# print('number of unique reblogged texts: {0}'.format(len(re_texts)))
# print('number of unique string in all texts: {0}'.format(len(set(df.text))))

# extra_stopwords = ['isnt','want','cant','wanna','im','could','ive','would','dont','get','also','us','thats','got','ur','wanted',
#                    'may', 'the', 'just', 'can', 'think', 'damn', 'still', 'guys', 'literally', 'hopefully', 'much', 'even', 'rly', 'guess', 'anon']#anything with a length of one
                   

# '''pre-processing'''
# # original_doc_set = list(df.text[df.photo=='no'])
# sel_df = df[df.photo=='no'] #extracting only-text posts
# doc_list = list(sel_df.text)


# ##############reading AP corpus
# text_df = ap_corpus('./data/ap.txt')
# doc_list = list(text_df.text_cleaned)

##############reading Newsgroup corpus
# text_df = newsgroup('./data/20newsgroup_preprocessed.csv')
# doc_list = list(text_df.text_cleaned)

##############Covid Tweet corpus
doc_list=[]
with open('./data/covid_tweets','r',encoding='utf-8') as txtfile:
    doc_list = txtfile.readlines()


#loading ref corpus for coherene score for lda_mallet
wiki_docs = loading_wiki_docs('./data/wiki_sampled_5p.txt')
#doing pre-processing on wiki-pedia documents
pre_processed_wiki, _ = preprocess_data(wiki_docs)
wiki_vocab_dict, _ = prepare_corpus(pre_processed_wiki)
del wiki_docs

In [54]:
#removing terms that are not in Wikipedia ref-corpus
pre_processed_docs,filtered_docs = preprocess_data(doc_list,extra_stopwords={})
#generate vocabulary and texts
vocab_dict, doc_term_matrix = prepare_corpus(pre_processed_docs)

#finding stopwords that are not in Wikipedia and removing those
extra_stopwords = set(vocab_dict.token2id.keys()).difference(set(wiki_vocab_dict.token2id.keys()))
pre_processed_docs,filtered_docs = preprocess_data(doc_list,extra_stopwords=extra_stopwords)
vocab_dict, doc_term_matrix = prepare_corpus(pre_processed_docs)

In [ ]:
#running many many LDA and storing their pair of terms

start = 36; limit=68; step = 1
runs = 3
term_pairs = set()

for num_topics in tqdm(range(start, limit+1, step)):
    model_t = []
    purity_t = []
    coherence_t = []
    contrast_t = []
    for r in range(runs):
        #model=LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model = LdaMallet(mallet_path, corpus=doc_term_matrix, num_topics=num_topics, id2word=vocab_dict,optimize_interval = 25,workers=1)

        #storing top_terms
        for tn in range(num_topics): 
            tt = model.show_topic(tn,topn=20)

            #saving top_terms and their counts
            top_terms = [i[0] for i in tt]

            #making pair terms
            for i in range(len(top_terms)):
                for j in range(i+1,len(top_terms)):
                    term_pairs.add((top_terms[i],top_terms[j]))
    

  0%|                                                    | 0/33 [00:00<?, ?it/s]Mallet LDA: 36 topics, 6 topic bits, 111111 topic mask
Data loaded.
max tokens: 35
total tokens: 3088644
<10> LL/token: -10.72143
<20> LL/token: -9.83314
<30> LL/token: -9.17704
<40> LL/token: -8.78943

0	1.38889	lives life cancer vulnerable issue spreading illness transmission shows analysis normal severe die prevent vaccines misinformation problems investigation folks release 
1	1.38889	analytics latest data team county insights population confirmed day growth deaths cases daily distribution total state tn dc fl wa 
2	1.38889	care workers hiv supply health wastewater gop access dr water highly mental big sites surveillance issues provide helping tax nose 
3	1.38889	health coronavirus today patients corona vaccine policy medicine doctor war nhs hospital insurance climate pharmaceutical biotech solution medical hospitals treatment 
4	1.38889	china cases shanghai reported infections city local chinese lockdo

<100> LL/token: -8.31774
<110> LL/token: -8.30308
<120> LL/token: -8.28948
<130> LL/token: -8.27958
<140> LL/token: -8.27221

0	1.38889	life lives vulnerable lost back spreading normal save left wrong heart line indian matter giving times failed folks investigation humanity 
1	1.38889	analytics data latest team county insights population confirmed day growth cases deaths distribution daily total state mn fl wa nc 
2	1.38889	support access hiv policies supply work communities wastewater issues provide businesses water surveillance investment employees network helping needed tax police 
3	1.38889	today patients coronavirus health corona hospital policy medicine doctor war nhs vaccine care hospitals insurance climate pharmaceutical biotech medical solution 
4	1.38889	china cases shanghai reported infections city local chinese lockdown beijing outbreak confirmed monday wednesday thursday tuesday morning asymptomatic sunday thailand 
5	1.38889	vaccination vaccine doses pm age india vaccines

<200> LL/token: -8.23995
<210> LL/token: -8.23667
<220> LL/token: -8.23546
[beta: 0.00811] 
<230> LL/token: -8.1978
<240> LL/token: -8.19778

0	1.2539	life lives back spreading normal lost left save wrong line act matter times giving fast failed reason investigation folks humanity 
1	1.24191	analytics data latest team county insights population confirmed day growth cases deaths distribution daily total state wa fl id nc 
2	1.23729	support access work communities issues hiv policies supply provide wastewater vulnerable needed shows water helping continues problems small network tax 
3	1.14899	coronavirus patients today health corona hospital policy medicine doctor war nhs vaccine hospitals insurance climate pharmaceutical biotech care doctors medical 
4	1.19736	china shanghai cases reported city infections local chinese lockdown outbreak beijing monday wednesday tuesday thursday asymptomatic confirmed sunday amid national 
5	1.16573	vaccination vaccine doses pm india age st administered

[beta: 0.00712] 
<300> LL/token: -8.13181
<310> LL/token: -8.13291
<320> LL/token: -8.14101
[beta: 0.00722] 
<330> LL/token: -8.11648
<340> LL/token: -8.12598

0	0.75966	life lives back normal big left lost save spreading wrong times stop act line govt matter misinformation control point folks 
1	0.60775	analytics data latest team county insights population confirmed day cases deaths growth daily distribution total state death wa fl ny 
2	0.69696	support access work communities hiv policies issues supply vulnerable provide funding services affected continues helping wastewater needed investment small network 
3	0.49381	coronavirus patients today health hospital policy medicine corona doctor war nhs vaccine hospitals insurance pharmaceutical biotech care climate medical healthcare 
4	0.60445	china shanghai cases city reported local lockdown chinese infections beijing outbreak confirmed wednesday asymptomatic monday tuesday amid authorities saturday thursday 
5	0.54541	vaccination vaccin

[beta: 0.00788] 
<400> LL/token: -8.09154
<410> LL/token: -8.09369
<420> LL/token: -8.10389
[beta: 0.00826] 
<430> LL/token: -8.08057
<440> LL/token: -8.09459

0	0.42423	life lives made big normal stop save left back lost times wrong control matter good country spread misinformation govt point 
1	0.2446	analytics data latest team county insights population confirmed cases day deaths growth daily distribution total state death fl nc ny 
2	0.34914	support access work communities services hiv supply funding vulnerable digital policies provide part program including affected needed ensure monitoring response 
3	0.19442	patients health coronavirus hospital medicine today doctor nhs policy corona vaccine war insurance pharmaceutical biotech hospitals healthcare medical climate care 
4	0.27547	china shanghai cases lockdown city reported local chinese confirmed beijing infections outbreak asymptomatic amid residents tuesday taiwan monday authorities commission 
5	0.2388	vaccination vaccine dos

...

[beta: 0.01316] 
<600> LL/token: -8.0096
<610> LL/token: -7.9918
<620> LL/token: -7.99494
[beta: 0.01409] 
<630> LL/token: -7.96083
<640> LL/token: -7.96394

0	0.1307	amp people trump government media biden science stop dead gop lies million american americans died lives made real truth democrats 
1	0.04074	taste coronavirus real bad twitter time things sense buy common smell venom flu vma low de mortality facebook data loss 
2	0.08381	vaccines pandemic vaccine amp make countries global access time market making future work treatments world big health made company evidence 
3	0.06953	cases day rate deaths case week rates death data numbers high coronavirus level change days average daily current higher population 
4	0.02936	coronavirus health medicine doctor vaccine today patients nhs insurance biotech policy pharmaceutical gt healthcare medical doctors hopkins johns virus hospital 
5	0.02028	ukraine world russia music art love beat pandemic trending hope wave ehr amp final news emr li

[beta: 0.01687] 
<700> LL/token: -7.89643
<710> LL/token: -7.87451
<720> LL/token: -7.87524
[beta: 0.01777] 
<730> LL/token: -7.84666
<740> LL/token: -7.84766

0	0.08873	amp people trump pandemic government media science biden world stop lives died time dead gop made real lies make americans 
1	0.01184	taste coronavirus smell buy venom vma bad real de harry low mortality google loss styles twitter lisa barcelona impact bunny 
2	0.04183	vaccines vaccine pandemic market global amp countries make access billion world healthcare company big time companies treatments made china supply 
3	0.03969	cases deaths rate day case coronavirus week high numbers daily level states death change average rates data population reported days 
4	0.01304	coronavirus health medicine doctor vaccine patients insurance nhs biotech today policy pharmaceutical gt healthcare hopkins johns medical virus doctors pandemic 
5	0.00977	ukraine world music art love beat russia pandemic healthcare coronavirus wave ehr news

[beta: 0.02018] 
<800> LL/token: -7.79458
<810> LL/token: -7.77822
<820> LL/token: -7.77667
[beta: 0.02079] 
<830> LL/token: -7.75996
<840> LL/token: -7.75853

0	0.06698	amp people pandemic trump government vaccine world media stop time biden science public lives years virus vaccines died dead made 
1	0.00533	taste coronavirus venom buy vma bad real harry mortality de smell styles low lisa bunny impact barcelona eff guayaquil dylan 
2	0.02345	china market pandemic global vaccine vaccines billion economy countries amp year growth business supply demand oil company world coronavirus companies 
3	0.02496	cases deaths rate day case coronavirus week daily high million numbers reported data death average states change population level number 
4	0.00768	coronavirus health medicine doctor vaccine patients insurance nhs biotech today pharmaceutical policy gt healthcare hopkins johns medical virus pandemic vaccines 
5	0.00569	world ukraine music art love beat russia coronavirus healthcare pandem

[beta: 0.02247] 
<900> LL/token: -7.72575
<910> LL/token: -7.7156
<920> LL/token: -7.71587
[beta: 0.02286] 
<930> LL/token: -7.70566
<940> LL/token: -7.70356

0	0.0554	amp people pandemic trump government vaccine world public media science time vaccines stop lives biden virus years died china made 
1	0.00372	taste coronavirus venom vma buy bad real harry mortality styles de lisa bunny impact barcelona guayaquil low eff dylan marvel 
2	0.01662	china pandemic global market economy supply amp year business growth billion inflation coronavirus vaccine demand economic prices oil due world 
3	0.01829	cases deaths rate day coronavirus case week daily million reported data high average states numbers death population number change fatality 
4	0.00549	coronavirus health medicine doctor vaccine patients insurance nhs biotech today pharmaceutical policy gt healthcare hopkins johns virus medical pandemic vaccines 
5	0.0041	world ukraine music art love beat healthcare pandemic coronavirus russia eh

[beta: 0.02394] 
<1000> LL/token: -7.68299

Total time: 8 minutes 13 seconds
  6%|██▎                                    | 2/33 [51:02<13:14:46, 1538.27s/it]Mallet LDA: 38 topics, 6 topic bits, 111111 topic mask
Data loaded.
max tokens: 35
total tokens: 3088644
<10> LL/token: -10.74819
<20> LL/token: -9.80503
<30> LL/token: -9.12982
<40> LL/token: -8.75709


In [46]:
#converting set to list
term_pairs_ls = [[i[0],i[1]] for i in list(term_pairs)]

In [48]:
#computing Coherence for all term pairs
cscore = CoherenceModel(topics=term_pairs,dictionary=wiki_vocab_dict,texts=pre_processed_wiki,coherence='c_npmi',processes=3,topn=2).get_coherence_per_topic()


In [51]:
#Loading the DB
npmi_db = db('./db/wiki_5p_old')


Load NPMI coherence DB. 
Number of keys : 160476


In [50]:
#writing into the DB
for i in tqdm(range(len(term_pairs_ls))):
    #if the key does not exist, insert it
    try:
        npmi_db.db[(term_pairs_ls[i][0],term_pairs_ls[i][1])]
    except:
        npmi_db.db[(term_pairs_ls[i][0],term_pairs_ls[i][1])] = cscore[i]
    #do the other combination
    try:
        npmi_db.db[(term_pairs_ls[i][1],term_pairs_ls[i][0])]
    except:
        npmi_db.db[(term_pairs_ls[i][1],term_pairs_ls[i][0])] = cscore[i]

100%|███████████████████████████████████| 21005/21005 [00:10<00:00, 2029.64it/s]


In [28]:
# for k in tqdm(npmi_db.db.iterkeys()):
#     try:
#         npmi_db.db[(k[1],k[0])]
#     except:
#         npmi_db.db[(k[1],k[0])] =  npmi_db.db[k]

125087it [00:23, 5268.58it/s]
